In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as seabornInstance
from sklearn.model_selection import train_test_split
from sklearn.model_selection import LeaveOneOut
from sklearn import svm
from sklearn import metrics
from sklearn.metrics import confusion_matrix
from sklearn import preprocessing
from pathlib import Path
%matplotlib inline

In [6]:
data = pd.read_csv(r"C:\Users\sshin\Desktop\SYSHIN0228\RERC_STARTS_Sensor_Analysis\Machine_Learning_6MWT\Feature_Matrix_Prediction_6MWT.csv", sep=',')
data = data.dropna()

CM = np.zeros((2,2))
print(CM)

for n in [360]:
    data_filter = data.query('cut_off_time == @n')
#     print(data_filter)
    
    X = data_filter[[
        'FIM_AD', 'BBS_AD', 'MWT10_SSV_AD', 'MWT6_AD', 'TUG_AD',
        'AoM_Pel_tilt', 'AoM_Pel_oblq', 'AoM_Pel_ro', 'AoM_Pel_norm',
        'AoM_Ankle_US_x', 'AoM_Ankle_US_y', 'AoM_Ankle_US_z', 'AoM_Ankle_US_norm',
        'AoM_Ankle_AS_x', 'AoM_Ankle_AS_y', 'AoM_Ankle_AS_z', 'AoM_Ankle_AS_norm',
#         'SC_Gyr_norm_mean', 'DLS_R_Gyr_norm_mean', 'DLS_L_Gyr_norm_mean',
#         'SC_Gyr_norm_range', 'DLS_R_Gyr_norm_range', 'DLS_L_Gyr_norm_range',
#         'SC_Gyr_norm_rms', 'DLS_R_Gyr_norm_rms', 'DLS_L_Gyr_norm_rms',
#         'SC_Gyr_norm_std', 'DLS_R_Gyr_norm_std', 'DLS_L_Gyr_norm_std',
#         'SC_Gyr_norm_skew', 'DLS_R_Gyr_norm_skew', 'DLS_L_Gyr_norm_skew',
#         'SC_Gyr_norm_kurtosis', 'DLS_R_Gyr_norm_kurtosis', 'DLS_L_Gyr_norm_kurtosis',
#         'SC_Gyr_norm_SamEn', 'DLS_R_Gyr_norm_SamEn', 'DLS_L_Gyr_norm_SamEn',
#         'SC_Acc_norm_mean', 'DLS_R_Acc_norm_mean', 'DLS_L_Acc_norm_mean',
#         'SC_Acc_norm_range', 'DLS_R_Acc_norm_range', 'DLS_L_Acc_norm_range',
#         'SC_Acc_norm_rms', 'DLS_R_Acc_norm_rms', 'DLS_L_Acc_norm_rms',
#         'SC_Acc_norm_std', 'DLS_R_Acc_norm_std', 'DLS_L_Acc_norm_std',
#         'SC_Acc_norm_skew', 'DLS_R_Acc_norm_skew', 'DLS_L_Acc_norm_skew',
#         'SC_Acc_norm_kurtosis', 'DLS_R_Acc_norm_kurtosis', 'DLS_L_Acc_norm_kurtosis',
#         'SC_Acc_norm_SamEn', 'DLS_R_Acc_norm_SamEn', 'DLS_L_Acc_norm_SamEn'
                ]].values
    
    
    
#     y = data_filter['Ambul'].values.reshape(-1,1)


    # To combine Moderate and Severe
    k = 0
    for i in data_filter['Ambul']:
        if i == 'Moderate':
            data_filter['Ambul'].values[k] = 'Severe'
        k+=1
            
    print(data_filter['Ambul'])

    factor = pd.factorize(data_filter['Ambul'])
    data_filter.Ambul = factor[0]
    definitions = factor[1]
    print(data_filter.Ambul.head())
    print(definitions)
    

    min_max_scaler = preprocessing.MinMaxScaler()
    X_scale = min_max_scaler.fit_transform(X)
    y = data_filter['Ambul'].values


    print(X_scale)
    print(y)
    
    loo = LeaveOneOut()
    loo.get_n_splits(X)
    i = 0

    correct = 0
    incorrect = 0
    for train_index, test_index in loo.split(X):
#         print("TRAIN:", train_index, "TEST:", test_index)
        X_train, X_test = X_scale[train_index], X_scale[test_index]
        y_train, y_test = y[train_index], y[test_index]

        clf = svm.SVC(kernel = 'linear', C = 1)
        
        clf.fit(X_train,y_train)

        y_pred = clf.predict(X_test)
        
        reversefactor = dict(zip(range(2),definitions))
        y_test = np.vectorize(reversefactor.get)(y_test)
        y_pred = np.vectorize(reversefactor.get)(y_pred)     

#         print(y_pred)
#         df = pd.DataFrame({'Actual': y_test.flatten(), 'Predicted': y_pred.flatten()})       
#         confusion = pd.crosstab(y_pred,y_test,colnames=['Actual'], rownames=['Predicted'])
#         print(confusion)
#         print(confusion_matrix(y_test, y_pred))

#         print(df)

        print(y_pred)
        print(y_test)

        if y_test == ['Mild'] and y_pred == ['Mild']:
            CM[0][0] += 1
        elif y_test == ['Mild'] and y_pred == ['Severe']:
            CM[0][1] += 1
        elif y_test == ['Severe'] and y_pred == ['Severe']:
            CM[1][1] += 1
        elif y_test == ['Severe'] and y_pred == ['Mild']:
            CM[1][0] += 1
            
        print(CM)
        
            



        if y_test == y_pred:
            correct = correct + 1
        else:
            incorrect = incorrect + 1

        i = i+1
        print("Iteration:", i)  

    print("Correct: ", correct)
    print("Incorrect: ", incorrect)
    

[[0. 0.]
 [0. 0.]]
551      Mild
553      Mild
554      Mild
555      Mild
558      Mild
559    Severe
560      Mild
561      Mild
562      Mild
563    Severe
564    Severe
565      Mild
566      Mild
567      Mild
568    Severe
569      Mild
570    Severe
572    Severe
573    Severe
575      Mild
576      Mild
577    Severe
578      Mild
580    Severe
582    Severe
583      Mild
585    Severe
586    Severe
587      Mild
588      Mild
590      Mild
594      Mild
595      Mild
596      Mild
597    Severe
599    Severe
600      Mild
601    Severe
602      Mild
603      Mild
604      Mild
Name: Ambul, dtype: object
551    0
553    0
554    0
555    0
558    0
Name: Ambul, dtype: int64
Index(['Mild', 'Severe'], dtype='object')
[[8.08823529e-01 6.73076923e-01 2.73941068e-01 4.18282434e-01
  8.58091999e-02 4.69721250e-01 4.80947172e-01 7.12788670e-01
  6.18692017e-01 5.07588728e-01 3.44158554e-01 5.43536111e-01
  5.49700179e-01 4.84447820e-01 3.61969783e-01 5.12917266e-01
  5.56942362e-01]
 

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\generic.py:5208: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value
